<a href="https://colab.research.google.com/github/xnileshtiwari/AI-Fusion/blob/main/Descript_like_wordhighlights_subtitles/Descript_like_wordhighlights_subtitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code and files for this notebook: https://github.com/ramsrigouthamg/Supertranslate.ai/tree/main/Descript_like_wordhighlights_subtitles

Author: [Ramsri Goutham](https://twitter.com/ramsri_goutham)

In [1]:
!pip install --quiet ipython-autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.4 MB/s eta 0:00:00


In [2]:
%load_ext autotime

time: 252 µs (started: 2025-04-06 13:12:16 +00:00)


## Step1: Download video or upload the video in mp4 format that you want to convert to fancy word-level subtitles (audiogram)

In [ ]:
import os

mp4videoURL = "https://github.com/ramsrigouthamg/Supertranslate.ai/raw/main/Descript_like_wordhighlights_subtitles/Intro.mp4"
videofilename = mp4videoURL.split('/')[-1]
print (videofilename)


Intro.mp4


In [ ]:
import urllib.request
urllib.request.urlretrieve(mp4videoURL, videofilename)

('Intro.mp4', <http.client.HTTPMessage at 0x7d60d6e19710>)

In [3]:
videofilename = "/content/Untitled video - Made with Clipchamp.mp4"

time: 446 µs (started: 2025-04-06 13:12:51 +00:00)


## Step2: Extract audio (mp3) from the video file

In [4]:
!pip install ffmpeg-python==0.2.0

time: 2.21 s (started: 2025-04-06 13:12:56 +00:00)


In [7]:
import ffmpeg

audiofilename = videofilename.replace(".mp4",'.mp3')

# Create the ffmpeg input stream
input_stream = ffmpeg.input(videofilename)

# Extract the audio stream from the input stream
audio = input_stream.audio

# Save the audio stream as an MP3 file
output_stream = ffmpeg.output(audio, audiofilename)

# Overwrite output file if it already exists
output_stream = ffmpeg.overwrite_output(output_stream)

ffmpeg.run(output_stream)

print (audiofilename)


/content/Untitled video - Made with Clipchamp.mp3
time: 1.18 s (started: 2025-04-06 13:14:18 +00:00)


In [8]:
from IPython.display import Audio

Audio(audiofilename)


time: 13.3 ms (started: 2025-04-06 13:14:30 +00:00)


## Step3: Get word-level transcription (speech-to-text) from audio using OpenAI's Whisper

In [10]:
!pip install git+https://github.com/openai/whisper.git@248b6cb124225dd263bb9bd32d060b6517e067f8 -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.4 MB/s et

In [11]:
# Downgrade NumPy to a version compatible with your PyTorch
!pip uninstall -y numpy
!pip install numpy==1.24.3  # A stable 1.x version

# Restart the runtime after this installation

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 84.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.0.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 re

time: 8.34 s (started: 2025-04-06 13:17:54 +00:00)


In [2]:
audiofilename = "/content/Untitled video - Made with Clipchamp.mp3"

In [3]:
import whisper
                                          #might take some time (approx 3- 5min depending on audio length)
model = whisper.load_model("small")
result = model.transcribe(audiofilename,word_timestamps=True)
print(result)

{'text': ' People debate intelligence, for example, right? We talk about IQ tests and all that, but I think the only true test of intelligence is if you get what you want out of life, and there are two parts to that. One is getting what you want. So you know how to get it, and the second is wanting the right things, knowing what to want in the first place.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.2, 'end': 4.28, 'text': ' People debate intelligence, for example, right? We talk about IQ tests and all that, but I think', 'tokens': [50364, 3432, 7958, 7599, 11, 337, 1365, 11, 558, 30, 492, 751, 466, 28921, 6921, 293, 439, 300, 11, 457, 286, 519, 50578], 'temperature': 0.0, 'avg_logprob': -0.2252925526012074, 'compression_ratio': 1.6926829268292682, 'no_speech_prob': 0.04295175150036812, 'words': [{'word': ' People', 'start': 0.2, 'end': 0.6, 'probability': 0.4013230502605438}, {'word': ' debate', 'start': 0.6, 'end': 0.82, 'probability': 0.9962783455848694}, {'word': ' intelligence,

In [5]:
# total text
print (result['text'])

 People debate intelligence, for example, right? We talk about IQ tests and all that, but I think the only true test of intelligence is if you get what you want out of life, and there are two parts to that. One is getting what you want. So you know how to get it, and the second is wanting the right things, knowing what to want in the first place.


In [6]:
# each subtitle segment
for i in result['segments']:
  print (i)

{'id': 0, 'seek': 0, 'start': 0.2, 'end': 4.28, 'text': ' People debate intelligence, for example, right? We talk about IQ tests and all that, but I think', 'tokens': [50364, 3432, 7958, 7599, 11, 337, 1365, 11, 558, 30, 492, 751, 466, 28921, 6921, 293, 439, 300, 11, 457, 286, 519, 50578], 'temperature': 0.0, 'avg_logprob': -0.2252925526012074, 'compression_ratio': 1.6926829268292682, 'no_speech_prob': 0.04295175150036812, 'words': [{'word': ' People', 'start': 0.2, 'end': 0.6, 'probability': 0.4013230502605438}, {'word': ' debate', 'start': 0.6, 'end': 0.82, 'probability': 0.9962783455848694}, {'word': ' intelligence,', 'start': 0.82, 'end': 1.34, 'probability': 0.9594983458518982}, {'word': ' for', 'start': 1.56, 'end': 1.6, 'probability': 0.9618090391159058}, {'word': ' example,', 'start': 1.6, 'end': 1.9, 'probability': 0.9988344311714172}, {'word': ' right?', 'start': 2.16, 'end': 2.28, 'probability': 0.9122424721717834}, {'word': ' We', 'start': 2.44, 'end': 2.48, 'probability': 

In [7]:
wordlevel_info = []

for each in result['segments']:
  words = each['words']
  for word in words:
    # print (word['word'], "  ",word['start']," - ",word['end'])
    wordlevel_info.append({'word':word['word'].strip(),'start':word['start'],'end':word['end']})


In [8]:
wordlevel_info

[{'word': 'People', 'start': 0.2, 'end': 0.6},
 {'word': 'debate', 'start': 0.6, 'end': 0.82},
 {'word': 'intelligence,', 'start': 0.82, 'end': 1.34},
 {'word': 'for', 'start': 1.56, 'end': 1.6},
 {'word': 'example,', 'start': 1.6, 'end': 1.9},
 {'word': 'right?', 'start': 2.16, 'end': 2.28},
 {'word': 'We', 'start': 2.44, 'end': 2.48},
 {'word': 'talk', 'start': 2.48, 'end': 2.66},
 {'word': 'about', 'start': 2.66, 'end': 2.8},
 {'word': 'IQ', 'start': 2.8, 'end': 3.06},
 {'word': 'tests', 'start': 3.06, 'end': 3.4},
 {'word': 'and', 'start': 3.4, 'end': 3.58},
 {'word': 'all', 'start': 3.58, 'end': 3.7},
 {'word': 'that,', 'start': 3.7, 'end': 3.86},
 {'word': 'but', 'start': 3.94, 'end': 4.02},
 {'word': 'I', 'start': 4.02, 'end': 4.14},
 {'word': 'think', 'start': 4.14, 'end': 4.28},
 {'word': 'the', 'start': 4.28, 'end': 5.38},
 {'word': 'only', 'start': 5.38, 'end': 5.6},
 {'word': 'true', 'start': 5.6, 'end': 5.84},
 {'word': 'test', 'start': 5.84, 'end': 6.1},
 {'word': 'of', '

## Step4: Store word-level timestamps into JSON file

In [9]:
import json
with open('data.json', 'w') as f:
    json.dump(wordlevel_info, f,indent=4)

## Step5: Correct and save JSON file if there are transcription errors or if you need to merge any words into phrases. Load the new edited JSON file with new word-level timestamps

In [10]:
import json

with open('data.json', 'r') as f:
    wordlevel_info_modified = json.load(f)


In [11]:
wordlevel_info_modified

[{'word': 'People', 'start': 0.2, 'end': 0.6},
 {'word': 'debate', 'start': 0.6, 'end': 0.82},
 {'word': 'intelligence,', 'start': 0.82, 'end': 1.34},
 {'word': 'for', 'start': 1.56, 'end': 1.6},
 {'word': 'example,', 'start': 1.6, 'end': 1.9},
 {'word': 'right?', 'start': 2.16, 'end': 2.28},
 {'word': 'We', 'start': 2.44, 'end': 2.48},
 {'word': 'talk', 'start': 2.48, 'end': 2.66},
 {'word': 'about', 'start': 2.66, 'end': 2.8},
 {'word': 'IQ', 'start': 2.8, 'end': 3.06},
 {'word': 'tests', 'start': 3.06, 'end': 3.4},
 {'word': 'and', 'start': 3.4, 'end': 3.58},
 {'word': 'all', 'start': 3.58, 'end': 3.7},
 {'word': 'that,', 'start': 3.7, 'end': 3.86},
 {'word': 'but', 'start': 3.94, 'end': 4.02},
 {'word': 'I', 'start': 4.02, 'end': 4.14},
 {'word': 'think', 'start': 4.14, 'end': 4.28},
 {'word': 'the', 'start': 4.28, 'end': 5.38},
 {'word': 'only', 'start': 5.38, 'end': 5.6},
 {'word': 'true', 'start': 5.6, 'end': 5.84},
 {'word': 'test', 'start': 5.84, 'end': 6.1},
 {'word': 'of', '

## Step6: Convert word-level timestamps JSON to line-level timestamps JSON. We want to display one line at a time and highlight words in it as they are spoken so we do this transformation.

In [27]:
def split_text_into_lines(data):

    MaxChars = 20
    #maxduration in seconds
    MaxDuration = 1.0
    #Split if nothing is spoken (gap) for these many seconds
    MaxGap = 1.5

    subtitles = []
    line = []
    line_duration = 0
    line_chars = 0


    for idx,word_data in enumerate(data):
        word = word_data["word"]
        start = word_data["start"]
        end = word_data["end"]

        line.append(word_data)
        line_duration += end - start

        temp = " ".join(item["word"] for item in line)


        # Check if adding a new word exceeds the maximum character count or duration
        new_line_chars = len(temp)

        duration_exceeded = line_duration > MaxDuration
        chars_exceeded = new_line_chars > MaxChars
        if idx>0:
          gap = word_data['start'] - data[idx-1]['end']
          # print (word,start,end,gap)
          maxgap_exceeded = gap > MaxGap
        else:
          maxgap_exceeded = False


        if duration_exceeded or chars_exceeded or maxgap_exceeded:
            if line:
                subtitle_line = {
                    "word": " ".join(item["word"] for item in line),
                    "start": line[0]["start"],
                    "end": line[-1]["end"],
                    "textcontents": line
                }
                subtitles.append(subtitle_line)
                line = []
                line_duration = 0
                line_chars = 0


    if line:
        subtitle_line = {
            "word": " ".join(item["word"] for item in line),
            "start": line[0]["start"],
            "end": line[-1]["end"],
            "textcontents": line
        }
        subtitles.append(subtitle_line)

    return subtitles

In [28]:
linelevel_subtitles = split_text_into_lines(wordlevel_info_modified)
print (linelevel_subtitles)

[{'word': 'People debate intelligence,', 'start': 0.2, 'end': 1.34, 'textcontents': [{'word': 'People', 'start': 0.2, 'end': 0.6}, {'word': 'debate', 'start': 0.6, 'end': 0.82}, {'word': 'intelligence,', 'start': 0.82, 'end': 1.34}]}, {'word': 'for example, right? We', 'start': 1.56, 'end': 2.48, 'textcontents': [{'word': 'for', 'start': 1.56, 'end': 1.6}, {'word': 'example,', 'start': 1.6, 'end': 1.9}, {'word': 'right?', 'start': 2.16, 'end': 2.28}, {'word': 'We', 'start': 2.44, 'end': 2.48}]}, {'word': 'talk about IQ tests and', 'start': 2.48, 'end': 3.58, 'textcontents': [{'word': 'talk', 'start': 2.48, 'end': 2.66}, {'word': 'about', 'start': 2.66, 'end': 2.8}, {'word': 'IQ', 'start': 2.8, 'end': 3.06}, {'word': 'tests', 'start': 3.06, 'end': 3.4}, {'word': 'and', 'start': 3.4, 'end': 3.58}]}, {'word': 'all that, but I think', 'start': 3.58, 'end': 4.28, 'textcontents': [{'word': 'all', 'start': 3.58, 'end': 3.7}, {'word': 'that,', 'start': 3.7, 'end': 3.86}, {'word': 'but', 'start

In [29]:
for line in linelevel_subtitles:
  json_str = json.dumps(line, indent=4)
  print(json_str)

{
    "word": "People debate intelligence,",
    "start": 0.2,
    "end": 1.34,
    "textcontents": [
        {
            "word": "People",
            "start": 0.2,
            "end": 0.6
        },
        {
            "word": "debate",
            "start": 0.6,
            "end": 0.82
        },
        {
            "word": "intelligence,",
            "start": 0.82,
            "end": 1.34
        }
    ]
}
{
    "word": "for example, right? We",
    "start": 1.56,
    "end": 2.48,
    "textcontents": [
        {
            "word": "for",
            "start": 1.56,
            "end": 1.6
        },
        {
            "word": "example,",
            "start": 1.6,
            "end": 1.9
        },
        {
            "word": "right?",
            "start": 2.16,
            "end": 2.28
        },
        {
            "word": "We",
            "start": 2.44,
            "end": 2.48
        }
    ]
}
{
    "word": "talk about IQ tests and",
    "start": 2.48,
    "end": 3.58,

In [14]:
for line in linelevel_subtitles:
  json_str = json.dumps(line, indent=4)
  print(json_str)

{
    "word": "People debate intelligence,",
    "start": 0.2,
    "end": 1.34,
    "textcontents": [
        {
            "word": "People",
            "start": 0.2,
            "end": 0.6
        },
        {
            "word": "debate",
            "start": 0.6,
            "end": 0.82
        },
        {
            "word": "intelligence,",
            "start": 0.82,
            "end": 1.34
        }
    ]
}
{
    "word": "for example, right? We talk about",
    "start": 1.56,
    "end": 2.8,
    "textcontents": [
        {
            "word": "for",
            "start": 1.56,
            "end": 1.6
        },
        {
            "word": "example,",
            "start": 1.6,
            "end": 1.9
        },
        {
            "word": "right?",
            "start": 2.16,
            "end": 2.28
        },
        {
            "word": "We",
            "start": 2.44,
            "end": 2.48
        },
        {
            "word": "talk",
            "start": 2.48,
        

## Step7: Use Moviepy to create an audiogram with word-level highlights as they are spoken

In [15]:
!pip install moviepy==2.0.0.dev2 -q
!pip install imageio==2.25.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 85.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires imageio!=2.35.0,>=2.33, but you have imageio 2.25.1 which is incompatible.


In [16]:
!apt install imagemagick -q

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  imagemagick-6.q16 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libwmflite-0.2-7 netpbm
  poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x imagemagick-doc
  autotrace cups-bsd | lpr | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer
  povray radiance sane-utils texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev
  inkscape poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The followin

In [17]:
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [34]:
from moviepy.editor import TextClip, CompositeVideoClip, ColorClip
import numpy as np


def create_caption(textJSON, framesize,font = "Helvetica-Bold",fontsize=40, color='white', bgcolor='orange'):
    wordcount = len(textJSON['textcontents'])
    full_duration = textJSON['end']-textJSON['start']

    word_clips = []
    xy_textclips_positions =[]

    x_pos = 0
    y_pos = 0
    # max_height = 0
    frame_width = framesize[0]
    frame_height = framesize[1]
    x_buffer = frame_width*1/10
    y_buffer = frame_height*1/5

    space_width = ""
    space_height = ""

    for index,wordJSON in enumerate(textJSON['textcontents']):
      duration = wordJSON['end']-wordJSON['start']
      word_clip = TextClip(wordJSON['word'], font = font,fontsize=fontsize, color=color).set_start(textJSON['start']).set_duration(full_duration)
      word_clip_space = TextClip(" ", font = font,fontsize=fontsize, color=color).set_start(textJSON['start']).set_duration(full_duration)
      word_width, word_height = word_clip.size
      space_width,space_height = word_clip_space.size
      if x_pos + word_width+ space_width > frame_width-2*x_buffer:
            # Move to the next line
            x_pos = 0
            y_pos = y_pos+ word_height+40

            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos+x_buffer,
                "y_pos": y_pos+y_buffer,
                "width" : word_width,
                "height" : word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos+x_buffer, y_pos+y_buffer))
            word_clip_space = word_clip_space.set_position((x_pos+ word_width +x_buffer, y_pos+y_buffer))
            x_pos = word_width + space_width
      else:
            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos+x_buffer,
                "y_pos": y_pos+y_buffer,
                "width" : word_width,
                "height" : word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos+x_buffer, y_pos+y_buffer))
            word_clip_space = word_clip_space.set_position((x_pos+ word_width+ x_buffer, y_pos+y_buffer))

            x_pos = x_pos + word_width+ space_width


      word_clips.append(word_clip)
      word_clips.append(word_clip_space)


    for highlight_word in xy_textclips_positions:

      word_clip_highlight = TextClip(highlight_word['word'], font = font,fontsize=fontsize, color=color,bg_color = bgcolor).set_start(highlight_word['start']).set_duration(highlight_word['duration'])
      word_clip_highlight = word_clip_highlight.set_position((highlight_word['x_pos'], highlight_word['y_pos']))
      word_clips.append(word_clip_highlight)

    return word_clips



In [35]:
videofilename = "/content/Untitled video - Made with Clipchamp.mp4"

In [36]:

from moviepy.editor import TextClip, CompositeVideoClip, concatenate_videoclips,VideoFileClip, ColorClip
frame_size = (1080,1080)

all_linelevel_splits=[]

for line in linelevel_subtitles:
  out = create_caption(line,frame_size)
  all_linelevel_splits.extend(out)


# Load the input video
input_video = VideoFileClip(videofilename)
# Get the duration of the input video
input_video_duration = input_video.duration
# Create a color clip with the given frame size, color, and duration
background_clip = ColorClip(size=frame_size, color=(0, 0, 0)).set_duration(input_video_duration)

# If you want to overlay this on the original video uncomment this and also change frame_size, font size and color accordingly.
final_video = CompositeVideoClip([input_video] + all_linelevel_splits)

#final_video = CompositeVideoClip([background_clip] + all_linelevel_splits)

# Set the audio of the final video to be the same as the input video
final_video = final_video.set_audio(input_video.audio)

# Save the final clip as a video file with the audio included
final_video.write_videofile("output.mp4", fps=24, codec="libx264", audio_codec="aac")



Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4


## Step8: Visualize the final Descript like audiogram video

In [37]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open("output.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)